In [12]:
import pandas as pd

# ------------------------------------------------------------
# 1. FUNCIONES BASE
# ------------------------------------------------------------

def cargar_csv(path):
    return pd.read_csv(path, encoding="utf-8-sig")


def normalizar_columnas(df):
    df = df.rename(columns={
        "Modalidad": "modalidad",
        "Escuela": "escuela_origen",
        "APELLIDOS Y NOMBRES": "nombres",
        "ESCUELA PROFESIONAL": "carrera_primera",
        "PUNTAJE FINAL": "puntaje",
        "MERITO E.P ALCANZA VACANTE": "merito",
        "MERITO": "merito",
        "OBSERVACIÓN": "observacion",
        "OBSERVACI&OacuteN": "observacion",
        "ESCUELA SEGUNDA OPCIÓN": "carrera_segunda",
        "ESCUELA SEGUNDA OPCIÃN": "carrera_segunda"
    })
    
    # Si no existe merito, crearlo vacío
    if "merito" not in df.columns:
        df["merito"] = None
    
    # Si no existe carrera_segunda, crearlo vacío
    if "carrera_segunda" not in df.columns:
        df["carrera_segunda"] = ""
    
    return df


def agregar_campos_generales(df, anio, periodo):
    df["anio"] = anio
    df["periodo"] = periodo
    return df


def detectar_ingreso(df):
    df["INGRESA"] = df["merito"].notna().astype(int)

    def opcion(row):
        if row["INGRESA"] == 1:
            if pd.isna(row["carrera_primera"]) or row["carrera_primera"] == "":
                return "SEGUNDA"
            return "PRIMERA"
        return "NINGUNA"

    df["OPCION_INGRESO"] = df.apply(opcion, axis=1)
    return df


def generar_carrera_final(df):
    def carrera_final(row):
        if row["OPCION_INGRESO"] == "SEGUNDA" and isinstance(row["carrera_segunda"], str):
            return row["carrera_segunda"]
        return row["carrera_primera"]

    df["CARRERA_FINAL"] = df.apply(carrera_final, axis=1)
    return df


# ------------------------------------------------------------
# 2. ARCHIVOS A PROCESAR
# ------------------------------------------------------------

archivos = {
    "Resultados_Admision_2024I.csv": (2024, "I"),
    "Resultados_Admision_2024II.csv": (2024, "II"),
    "Resultados_Admision_2025I.csv": (2025, "I"),
    "Resultados_Admision_2025II.csv": (2025, "II"),
    "Resultados_Admision_2026I.csv": (2026, "I")
}

dfs = []

